In [22]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [23]:

df_final = pd.read_csv('bds_clean_scaled_encoded.csv')  # đọc lại dữ liệu đã làm sẵn


# ----------------------------------------------------------------------
# 1. XÂY DỰNG BIẾN MỤC TIÊU (Y)
# ----------------------------------------------------------------------


In [24]:


# Kiểm tra cột Gia_VND
if 'Gia_VND' not in df_final.columns:
    raise ValueError("ERROR: Không tìm thấy cột 'Gia_VND' trong df_final!")

# Tạo biến mục tiêu nhị phân 'High_Value' (1 nếu giá > median, 0 nếu giá <= median)
median_scaled_price = df_final['Gia_VND'].median() 
df_final['High_Value'] = (df_final['Gia_VND'] > median_scaled_price).astype(int)

# Kiểm tra phân phối classes
value_counts = df_final['High_Value'].value_counts()
print("\nPhân phối classes:")
print(value_counts)
print(f"Tỷ lệ class 1: {(value_counts[1] / len(df_final)):.2%}")



Phân phối classes:
High_Value
0    1269
1     747
Name: count, dtype: int64
Tỷ lệ class 1: 37.05%


In [25]:

# Loại bỏ cột Gia_VND và các cột thô không cần thiết
cols_to_drop = ['Gia_VND', 'Link', 'Ngày đăng', 'Loại hình căn hộ'] 
X = df_final.drop(columns=[col for col in cols_to_drop if col in df_final.columns] + ['High_Value'])
Y = df_final['High_Value']

print(f"\nKích thước DataFrame X (Đặc trưng): {X.shape}")
print(f"Kích thước Series Y (Mục tiêu): {Y.shape}")



Kích thước DataFrame X (Đặc trưng): (2016, 8)
Kích thước Series Y (Mục tiêu): (2016,)



# ----------------------------------------------------------------------
# 2. CHIA DỮ LIỆU VÀ HUẤN LUYỆN MÔ HÌNH
# ----------------------------------------------------------------------


In [26]:

# Kiểm tra điều kiện cho stratify
if len(Y.unique()) < 2:
    print("⚠️ Cảnh báo: Y chỉ có 1 class, không thể dùng stratify!")
    stratify = None
else:
    class_counts = Y.value_counts()
    if any(class_counts < 2):
        print("⚠️ Cảnh báo: Có class chỉ có 1 mẫu, không thể dùng stratify!")
        stratify = None
    else:
        stratify = Y

# Chia dữ liệu train/test
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=stratify
)

print("\nPhân phối classes trong tập train:")
print(pd.Series(Y_train).value_counts(normalize=True))
print("\nPhân phối classes trong tập test:")
print(pd.Series(Y_test).value_counts(normalize=True))



Phân phối classes trong tập train:
High_Value
0    0.629653
1    0.370347
Name: proportion, dtype: float64

Phân phối classes trong tập test:
High_Value
0    0.628713
1    0.371287
Name: proportion, dtype: float64


In [27]:

# Khởi tạo và huấn luyện mô hình
model = LogisticRegression(random_state=42, solver='liblinear')
model.fit(X_train, Y_train)

print("\n--- Huấn luyện Mô hình Logistic Regression HOÀN TẤT ---")
print(f"Kích thước tập huấn luyện: {X_train.shape[0]} mẫu")
print(f"Kích thước test : {Y_test.shape}")



--- Huấn luyện Mô hình Logistic Regression HOÀN TẤT ---
Kích thước tập huấn luyện: 1612 mẫu
Kích thước test : (404,)



# ----------------------------------------------------------------------
# 3. ĐÁNH GIÁ HIỆU SUẤT MÔ HÌNH
# ----------------------------------------------------------------------


In [28]:

Y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred, target_names=['Giá Thấp (0)', 'Giá Cao (1)'])

print(f"\nĐộ chính xác (Accuracy) trên tập kiểm tra: {accuracy:.4f}")
print("\nBáo cáo phân loại chi tiết (Classification Report):")
print(report)



Độ chính xác (Accuracy) trên tập kiểm tra: 0.7079

Báo cáo phân loại chi tiết (Classification Report):
              precision    recall  f1-score   support

Giá Thấp (0)       0.72      0.89      0.79       254
 Giá Cao (1)       0.68      0.41      0.51       150

    accuracy                           0.71       404
   macro avg       0.70      0.65      0.65       404
weighted avg       0.70      0.71      0.69       404



In [29]:

# Hiển thị feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': abs(model.coef_[0])
})
print("\nTop 10 đặc trưng quan trọng nhất:")
print(feature_importance.sort_values('Importance', ascending=False).head(10))


Top 10 đặc trưng quan trọng nhất:
                      Feature  Importance
2  Diện tích (m²) đã làm sạch    1.911970
1        Số phòng tắm/vệ sinh    1.572390
0                Số phòng ngủ    1.441620
5     Noi_that_Day du/Cao cap    1.009093
7       Phap_ly_So do/So hong    0.735401
6        Phap_ly_Khac/Chua co    0.731368
4             Noi_that_Co ban    0.574418
3            Noi_that_Chua ro    0.222065
